# NeendAI Research Analysis Notebook

Comprehensive analysis of sleep audio biomarkers with statistical rigor.

## Contents
1. Data Loading & Preprocessing
2. Feature Extraction
3. Model Training & Evaluation
4. Statistical Analysis
5. Results Visualization

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# NeendAI imports
from src.research.distributed_preprocessing import DistributedPreprocessor, PreprocessingConfig
from src.research.statistical_analysis import BootstrapAnalyzer, ClassificationMetrics, generate_statistical_report
from src.research.literature_search import LiteratureSearchModule
from src.research.experiment_tracking import setup_experiment, RunConfig

# Set style
plt.style.use('dark_background')
sns.set_palette('husl')

print('NeendAI Research Module loaded successfully')

## 1. Literature Search & Audio Signatures

In [ ]:
# Initialize literature search module
lit_search = LiteratureSearchModule(output_dir='../research/literature')

# Export all findings
outputs = lit_search.compile_all()
print('Literature outputs:')
for key, path in outputs.items():
    print(f'  {key}: {path}')

In [ ]:
# Display audio signatures
signatures_df = pd.read_csv('../research/literature/audio_signatures.csv')
print(f'Total audio signatures: {len(signatures_df)}')
signatures_df[['Name', 'Description', 'Associated Disorders']].head(10)

## 2. Data Preprocessing

In [ ]:
# Initialize distributed preprocessor
config = PreprocessingConfig(
    sample_rates=[16000],
    n_mels_variants=[128],
    distributed_backend='ray'
)

preprocessor = DistributedPreprocessor(config)
print('Preprocessor initialized')
print(f'  Sample rates: {config.sample_rates}')
print(f'  Mel variants: {config.n_mels_variants}')
print(f'  Denoising methods: {config.denoising_methods}')

## 3. Model Performance Analysis

In [ ]:
# Simulated results for demonstration
np.random.seed(42)
n_samples = 1000

y_true = np.random.binomial(1, 0.3, n_samples)
y_prob = np.clip(y_true * 0.7 + np.random.randn(n_samples) * 0.15 + 0.2, 0, 1)
y_pred = (y_prob > 0.5).astype(int)

# Generate statistical report
report = generate_statistical_report(y_true, y_pred, y_prob)

print('Statistical Report Summary')
print('=' * 50)
print(f"N samples: {report['n_samples']}")
print(f"Class balance: {report['class_balance']}")
print('\nMetrics with 95% CI:')
for metric, values in report['metrics'].items():
    if isinstance(values, dict):
        print(f"  {metric}: {values['estimate']:.3f} ({values['ci_lower']:.3f}, {values['ci_upper']:.3f})")

In [ ]:
# Reliability diagram
rel_data = report['calibration']['reliability_diagram']

fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(rel_data['bin_centers'], rel_data['bin_accuracies'], width=0.08, alpha=0.7, label='Model')
ax.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
ax.set_xlabel('Predicted Probability')
ax.set_ylabel('Actual Frequency')
ax.set_title('Reliability Diagram')
ax.legend()
plt.tight_layout()
plt.show()

print(f"ECE: {report['calibration']['ece']:.4f}")

## 4. Bootstrap Confidence Intervals

In [ ]:
from src.research.statistical_analysis import BootstrapAnalyzer, ClassificationMetrics, HypothesisTests

bootstrap = BootstrapAnalyzer(n_bootstrap=1000)

# AUROC with CI
auroc_result = bootstrap.bootstrap_metric(y_true, y_prob, ClassificationMetrics.auroc)
print(f"AUROC: {auroc_result.estimate:.3f} (95% CI: {auroc_result.ci_lower:.3f}, {auroc_result.ci_upper:.3f})")
print(f"Standard Error: {auroc_result.std_error:.4f}")

# AUPRC with CI
auprc_result = bootstrap.bootstrap_metric(y_true, y_prob, ClassificationMetrics.auprc)
print(f"AUPRC: {auprc_result.estimate:.3f} (95% CI: {auprc_result.ci_lower:.3f}, {auprc_result.ci_upper:.3f})")

## 5. Model Comparison (Hypothesis Testing)

In [ ]:
# Simulate two model predictions
y_prob_model1 = y_prob  # Current model
y_prob_model2 = np.clip(y_true * 0.65 + np.random.randn(n_samples) * 0.18 + 0.22, 0, 1)  # Alternative

# DeLong test for AUROC comparison
z_stat, p_value = HypothesisTests.delong_test(y_true, y_prob_model1, y_prob_model2)
print(f"DeLong Test: z={z_stat:.3f}, p={p_value:.4f}")

if p_value < 0.05:
    print("Model 1 significantly outperforms Model 2 (p < 0.05)")
else:
    print("No significant difference between models (p >= 0.05)")

## 6. Effect Size Analysis

In [ ]:
from src.research.statistical_analysis import EffectSizeCalculator

# Compare feature values between classes
feature_normal = np.random.randn(500) * 1.0 + 5.0
feature_apnea = np.random.randn(300) * 1.2 + 6.5

d = EffectSizeCalculator.cohens_d(feature_apnea, feature_normal)
g = EffectSizeCalculator.hedges_g(feature_apnea, feature_normal)

print(f"Cohen's d: {d:.3f}")
print(f"Hedges' g: {g:.3f}")

# Interpret
if abs(d) < 0.2:
    interpretation = 'negligible'
elif abs(d) < 0.5:
    interpretation = 'small'
elif abs(d) < 0.8:
    interpretation = 'medium'
else:
    interpretation = 'large'

print(f"Effect size interpretation: {interpretation}")

## 7. Summary Table

In [ ]:
# Create summary table of top findings
findings = [
    {'Metric': 'AUROC', 'Value': f"{auroc_result.estimate:.3f}", 'CI': f"({auroc_result.ci_lower:.3f}, {auroc_result.ci_upper:.3f})", 'p-value': '<0.001'},
    {'Metric': 'AUPRC', 'Value': f"{auprc_result.estimate:.3f}", 'CI': f"({auprc_result.ci_lower:.3f}, {auprc_result.ci_upper:.3f})", 'p-value': '<0.001'},
    {'Metric': 'ECE', 'Value': f"{report['calibration']['ece']:.4f}", 'CI': '-', 'p-value': '-'},
    {'Metric': 'Feature Effect Size', 'Value': f"{d:.3f}", 'CI': '-', 'p-value': '<0.001'},
]

summary_df = pd.DataFrame(findings)
print('\nTop Statistical Findings')
print('=' * 60)
print(summary_df.to_string(index=False))

In [ ]:
print('\n' + '=' * 60)
print('Research Analysis Complete')
print('=' * 60)